### Randomly Sending SMS to Recipients via Twilio


In [ ]:
import os
import pandas as pd
import time
import random
from flask import Flask, render_template, request, redirect, url_for, flash
from twilio.rest import Client
from werkzeug.utils import secure_filename
import threading
from configparser import ConfigParser

# Loading configuration from config.ini
config = ConfigParser()
config.read('config.ini')

# Twilio credentials from config.ini
account_sid = config.get('twilio', 'account_sid')
auth_token = config.get('twilio', 'auth_token')
from_number = config.get('twilio', 'from_number')

# Time slot from config.ini
total_time_slot = config.getint('time_slot', 'total_time_slot')

# Initializing Twilio client
client = Client(account_sid, auth_token)

# Flask app Configuration
app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = 'uploaded_files'
os.makedirs(app.config['UPLOAD_FOLDER'], exist_ok=True)

# Sending SMS to all CSV Recipients
def send_sms_from_csv(file_path):
    df = pd.read_csv(file_path)
    num_records = len(df)
    delay_max = int(total_time_slot / num_records)
    delay_min = delay_max - 300

    for index, row in df.iterrows():
        phone_number = row['Last Name']
        link = row['Link']
        print(phone_number)
        print(link)
        if pd.isna(phone_number) or pd.isna(link):
            print(f"Skipping row {index} due to missing data. Phone: {phone_number}, Link: {link}")
            continue
        phone_number = str(phone_number).strip()
        link = str(link).strip()
        if not phone_number or len(phone_number) < 10:
            print(f"Skipping invalid phone number: {phone_number}")
            continue
        try:
            message = client.messages.create(
                body= "Follow this link to the Survey:\n" + link,
                from_=from_number,
                to='+1' + phone_number
            )
            print(f"Message sent to {phone_number}, SID: {message.sid}")
        except Exception as e:
            print(f"Failed to send message to {phone_number}: {e}")
        delay = random.randint(delay_min, delay_max)
        print(f"Waiting {delay} seconds before sending SMS to {phone_number}")
        time.sleep(delay)

# Loading CSV files from Folder
def load_and_send_sms(folder_path):
    csv_files = sorted([f for f in os.listdir(folder_path) if f.endswith('.csv')])
    for i, csv_file in enumerate(csv_files):
        file_path = os.path.join(folder_path, csv_file)
        print(f"Processing file: {file_path}")
        send_sms_from_csv(file_path)


@app.route('/')
def index():
    return '''
        <html>
            <head>
                <style>
                    body {
                        display: flex;
                        flex-direction: column;
                        align-items: center;
                        justify-content: center;
                        height: 100vh;
                        margin: 0;
                        font-family: 'Arial', sans-serif;
                        background-color: #f0f8ff;  /* Light background color */
                        color: #333;  /* Dark text color */
                    }
                    h1 {
                        margin-bottom: 20px;
                        font-size: 2.5em;
                        color: #0066cc;  /* Title color */
                    }
                    form {
                        background-color: #ffffff;  /* White background for form */
                        padding: 20px;
                        border-radius: 8px;  /* Rounded corners */
                        box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);  /* Soft shadow */
                        text-align: center;
                    }
                    label {
                        margin-right: 10px;
                        font-size: 1.2em;  /* Larger font for label */
                    }
                    input[type="file"] {
                        margin-top: 10px;
                        padding: 10px;  /* Padding for better appearance */
                        border: 2px solid #0066cc;  /* Border color */
                        border-radius: 5px;  /* Rounded corners */
                        font-size: 1em;  /* Font size */
                    }
                    button {
                        margin-top: 20px;
                        padding: 10px 20px;  /* Padding for button */
                        border: none;  /* No border */
                        border-radius: 5px;  /* Rounded corners */
                        background-color: #0066cc;  /* Button color */
                        color: white;  /* Text color */
                        font-size: 1.2em;  /* Font size */
                        cursor: pointer;  /* Pointer cursor on hover */
                        transition: background-color 0.3s;  /* Transition effect */
                    }
                    button:hover {
                        background-color: #005bb5;  /* Darker shade on hover */
                    }
                    .warning {
                        margin-top: 20px;
                        font-size: 1.1em;
                        color: #cc0000;  /* Warning color */
                        font-weight: bold;
                    }
                </style>
            </head>
            <body>
                <h1>AQI Survey</h1>
                <form action="/upload" method="post" enctype="multipart/form-data">
                    <label for="folder">Choose folder:</label>
                    <input type="file" name="folder" id="folder" webkitdirectory directory multiple>
                    <button type="submit">Start SMS sending</button>
                </form>
                <div class="warning">
                    Please upload CSV files only!
                </div>
            </body>
        </html>
    '''



@app.route('/upload', methods=['POST'])
def upload_folder():
    if 'folder' not in request.files:
        flash('No folder part')
        return redirect(url_for('index'))

    files = request.files.getlist('folder')
    folder_path = os.path.join(app.config['UPLOAD_FOLDER'], "csv_folder")

    # Clear old files in the upload folder
    if os.path.exists(folder_path):
        for old_file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, old_file)
            try:
                if os.path.isfile(file_path):
                    os.unlink(file_path)
            except Exception as e:
                print(f"Error deleting file {file_path}: {e}")

    os.makedirs(folder_path, exist_ok=True)

    # Save the new CSV files
    for file in files:
        if file.filename.endswith('.csv'):
            filename = secure_filename(file.filename)
            file.save(os.path.join(folder_path, filename))

    # Start SMS sending in a separate thread
    threading.Thread(target=load_and_send_sms, args=(folder_path,)).start()
    return redirect(url_for('index'))


# Run Flask app
def run_flask():
    app.run(debug=True, use_reloader=False)  # Disable reloader if running in Jupyter

# Start Flask in a new thread
threading.Thread(target=run_flask).start()


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [25/Oct/2024 22:12:25] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2024 22:12:33] "POST /upload HTTP/1.1" 302 -
127.0.0.1 - - [25/Oct/2024 22:12:33] "GET / HTTP/1.1" 200 -


Processing file: uploaded_files\csv_folder\Day_4_export-EMD_VTCkLXUzdbhJSA1-2024-10-25T15-42-19-989Z.csv
600
300
3134684490
https://umich.qualtrics.com/jfe/form/SV_89dTrDV9oauyh4a?Q_CHL=gl&Q_DL=EMD_VTCkLXUzdbhJSA1_89dTrDV9oauyh4a_CGC_LTFe6I2QJ1dVW1A&_g_=g
Waiting 434 seconds before sending SMS to 3134684490
